<a href="https://colab.research.google.com/github/Steven-Macas/PIA/blob/Red-Neuronal/stevenMacas_red_neuronal_b%C3%A1sica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Implementación de la red neuronal con retropropagación

In [ ]:
class NetNode(object):

  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

In [ ]:
class Network(object):

  def __init__(self, layers, activation = "relu"):
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    self.activation = activation #Elección de activación
    sizes = len(layers)
    for layer in range(1, sizes):
        for node in self.net[layer]:
            for unit in self.net[layer - 1]:
                node.inputs.append(unit)
                node.weights.append(0)

  def accuracy(self, examples):
    correct = 0
    for x_test, y_test in examples:
        prediction = self.predict(x_test)
        if (y_test[prediction] == 1):
            correct += 1
    return correct / len(examples)

  def backpropagation(self, eta, examples, epochs):
    inputs = self.net[0]
    outputs = self.net[-1]
    layer_size = len(self.net)
    for layer in self.net[1:]:
        for node in layer:
            node.weights = [np.random.uniform() for _ in range(len(node.weights))]
    for epoch in range(epochs):
        for x_train, y_train in examples:
            for value, node in zip(x_train, inputs):
                node.value = value
            for layer in self.net[1:]:
                for node in layer:
                    in_val = [n.value for n in node.inputs]
                    unit_value = np.dot(in_val, node.weights)
                    #activation
                    if self.activation == "relu":
                      node.value = self.relu(unit_value)
                    elif self.activation == "sigmoide":
                      node.value = self.sigmoide(unit_value)
            delta = [[] for _ in range(layer_size)]
            err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
            #Elección de activación
            if self.activation == "relu":
              delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            elif self.activation == "sigmoide":
              delta[-1] = [self.sigmoide_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            hidden_layers = layer_size - 2
            for i in range(hidden_layers, 0, -1):
                layer = self.net[i]
                n_layers = len(layer)
                w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                #Elección de activción
                if self.activation == "relu":
                  delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                elif self.activation == "sigmoide":
                  delta[i] = [self.sigmoide_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
            for i in range(1, layer_size):
                layer = self.net[i]
                in_val = [node.value for node in self.net[i - 1]]
                n_layers = len(self.net[i])
                for j in range(n_layers):
                    layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
        print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
  
  def predict(self, input_data):
    inputs = self.net[0]
    for v, n in zip(input_data, inputs):
        n.value = v
    for layer in self.net[1:]:
        for node in layer:
            in_val = [n.value for n in node.inputs]
            unit_value = np.dot(in_val, node.weights)
            node.value = self.relu(unit_value)
    outputs = self.net[-1]
    return outputs.index(max(outputs, key=lambda node: node.value))

  def relu(self, z):
    return max(0, z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
      return 1.0 / (1.0 + np.exp(-z))

  def sigmoide_prime(self, z):
      return self.sigmoide(z) * (1 - self.sigmoide(z))
  
  def set_weights(self, new_weights, new_biases):
    self.weights = new_weights
    self.biases = new_biases

  def weights(self):
    return [[[node.weights[i] for i in range(len(node.weights))] for node in layer] for layer in self.net]

# Usando la red neuronal con un dataset

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [ ]:
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [ ]:
iris_x_normalized = normalize(iris_X, axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

In [ ]:
examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

In [ ]:
net = Network([4, 7, 3])
net.backpropagation(0.1, examples, 500)

epoch 0/500 | total error=-0.0025177957013839494
epoch 1/500 | total error=-0.0020843343884047625
epoch 2/500 | total error=-0.002034818619924651
epoch 3/500 | total error=-0.0019993933170022052
epoch 4/500 | total error=-0.0019357355525257303
epoch 5/500 | total error=-0.0018497714661336122
epoch 6/500 | total error=-0.0017298002559102328
epoch 7/500 | total error=-0.0015744937469923633
epoch 8/500 | total error=-0.0014048534169371731
epoch 9/500 | total error=-0.0012084736408941542
epoch 10/500 | total error=-0.0009296759351382457
epoch 11/500 | total error=-0.0009794772682172714
epoch 12/500 | total error=-0.0010520793535985968
epoch 13/500 | total error=-0.0011033249174680234
epoch 14/500 | total error=-0.0011531872988391112
epoch 15/500 | total error=-0.0011996799858129182
epoch 16/500 | total error=-0.001254104658163025
epoch 17/500 | total error=-0.0013015520224873506
epoch 18/500 | total error=-0.0013441137031622913
epoch 19/500 | total error=-0.0013880270058002867
epoch 20/500

In [ ]:
#precisión alcanzada con los datos de entrenamiento
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.95


In [ ]:
#precisión alcanzada con los datos de prueba
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9666666666666667


In [ ]:
#probando con un dato
prediction = net.predict(X_test[2])
print(f"Desired output: {y_test[2]}")
print(f"Index of output: {prediction}")

Desired output: [1. 0. 0.]
Index of output: 0


# Modificación de la implementación de la red neuronal

In [ ]:
# agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas
def weights(self):
    return [[[node.weights[i] for i in range(len(node.weights))] for node in layer] for layer in self.net]
# agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas
def set_weights(self, new_weights, new_biases):
    self.weights = new_weights
    self.biases = new_biases

In [ ]:
# agrega los métodos sigmoide() y sigmoide_prime() a la clase Network
def sigmoide(self, z):
    return 1.0 / (1.0 + np.exp(-z))
def sigmoide_prime(self, z):
    return self.sigmoide(z) * (1 - self.sigmoide(z))

# modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()
#Línea 5,36,43,53

In [ ]:
# los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación

# modifícalos de tal manera que también se puedan utilizar con problemas de regresión
class Network(object):

  def __init__(self, layers, activation = "relu"):
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    self.activation = activation #Elección de activación
    sizes = len(layers)
    for layer in range(1, sizes):
        for node in self.net[layer]:
            for unit in self.net[layer - 1]:
                node.inputs.append(unit)
                node.weights.append(0)

  def accuracy(self, examples):
    total_error = 0
    for x_test, y_test in examples:
      prediction = self.predict(x_test)
      total_error += sum((y_test[i] - prediction[i])**2 for i in range(len(prediction)))
    return 1 - (total_error / len(examples))

  def backpropagation(self, eta, examples, epochs):
    inputs = self.net[0]
    outputs = self.net[-1]
    layer_size = len(self.net)
    for layer in self.net[1:]:
        for node in layer:
            node.weights = [np.random.uniform() for _ in range(len(node.weights))]
    node_values = [] #Almacenamos los valores de los nodos en cada capa para cada ejemplo
    for epoch in range(epochs):
        for x_train, y_train in examples:
            for value, node in zip(x_train, inputs):
                node.value = value
            for layer in self.net[1:]:
                for node in layer:
                    in_val = [n.value for n in node.inputs]
                    unit_value = np.dot(in_val, node.weights)
                    #activation
                    if self.activation == "relu":
                      node.value = self.relu(unit_value)
                    elif self.activation == "sigmoide":
                      node.value = self.sigmoide(unit_value)
            node_values.append([node.value for layer in self.net for node in layer]) # agregar valores de nodos actuales
            delta = [[] for _ in range(layer_size)]
            err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
            #Elección de activación
            if self.activation == "relu":
              delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            elif self.activation == "sigmoide":
              delta[-1] = [self.sigmoide_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
            hidden_layers = layer_size - 2
            for i in range(hidden_layers, 0, -1):
                layer = self.net[i]
                n_layers = len(layer)
                w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                #Elección de activción
                if self.activation == "relu":
                  delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                elif self.activation == "sigmoide":
                  delta[i] = [self.sigmoide_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
            for i in range(1, layer_size):
                layer = self.net[i]
                in_val = [node.value for node in self.net[i - 1]]
                n_layers = len(self.net[i])
                for j in range(n_layers):
                    layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
        print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
    return np.array(node_values)
  
  def predict(self, input_data):
    inputs = self.net[0]
    for v, n in zip(input_data, inputs):
        n.value = v
    for layer in self.net[1:]:
        for node in layer:
            in_val = [n.value for n in node.inputs]
            unit_value = np.dot(in_val, node.weights)
            node.value = self.relu(unit_value) if self.activation_func == 'relu' else self.sigmoide(unit_value)
    outputs = self.net[-1]
    return outputs[0].value

  def relu(self, z):
    return max(0, z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
      return 1.0 / (1.0 + np.exp(-z))

  def sigmoide_prime(self, z):
      return self.sigmoide(z) * (1 - self.sigmoide(z))
  
  def set_weights(self, new_weights, new_biases):
    self.weights = new_weights
    self.biases = new_biases

  def weights(self):
    return [[[node.weights[i] for i in range(len(node.weights))] for node in layer] for layer in self.net]

In [ ]:
# modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento
#Linea 30,44,69

In [ ]:
# una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])

In [ ]:
# ejecuta la red neuronal para los datos de ejemplo de los apuntes
# comprueba los valores de los nodos y de los pesos
# los valores de los nodos tienen que ser los mismos que los de los apuntes
# los valores de los pesos son ligeramente diferentes, ¿por qué?

net = Network([3, 4, 2])

net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# o
net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
net.weights()